In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from transformers import XLMRobertaTokenizer, XLMRobertaModel
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import os
from PIL import Image

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directories
image_dir = r"C:\Users\golla\Desktop\BTECH\8THSEM"
csv_path = "final_data.csv"

# Image Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [40]:
# Load Pre-trained ResNet-50 Model and Modify Last Layer
class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
        for param in self.resnet.parameters():
            param.requires_grad = False  # Freeze all layers except the last one
        self.resnet.fc = nn.Linear(2048, 2)  # Modify last layer for binary classification
    
    def forward(self, x):
        return self.resnet(x)

resnet_model = ResNetModel().to(device)
resnet_model.train()


ResNetModel(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
       

In [25]:
# Define Text Classifier Model
class TextClassifier(nn.Module):
    def __init__(self):
        super(TextClassifier, self).__init__()
        self.xlmr = xlmr_model
        for param in self.xlmr.parameters():
            param.requires_grad = False  # Freeze pre-trained layers
        self.fc = nn.Linear(768, 2)  # Modify last layer
    
    def forward(self, input_ids, attention_mask):
        outputs = self.xlmr(input_ids=input_ids, attention_mask=attention_mask)
        return self.fc(outputs.pooler_output)

text_model = TextClassifier().to(device)
text_model.train()

# Define Loss and Optimizer (Only Fine-Tuning Last Layer)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(list(resnet_model.resnet.fc.parameters()) + list(text_model.fc.parameters()), lr=0.0001)


In [26]:
# Quick Fine-Tuning for Few Iterations
def fine_tune(model, loader):
    model.train()
    for batch in loader:
        optimizer.zero_grad()
        inputs, masks, labels = batch
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)
        outputs = model(inputs, masks)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print("Fine-tuning complete.")

fine_tune(text_model, text_loader)  # Fine-tune for text
resnet_model.train()  # Ensure ResNet is in training mode for quick fine-tuning


Fine-tuning complete.


ResNetModel(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
       

In [47]:
# Inference Function
def predict(user_input):
    if os.path.exists(user_input):  # If input is an image
        img = Image.open(user_input).convert("RGB")
        img = transform(img).unsqueeze(0).to(device)
        
        resnet_model.eval()
        with torch.no_grad():
            pred = resnet_model(img).argmax(1).item()
        return "Fake News" if pred == 1 else "Real News"
    else:  # If input is text
        encoding = tokenizer(user_input, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)
        
        text_model.eval()
        with torch.no_grad():
            pred = text_model(encoding['input_ids'], encoding['attention_mask']).argmax(1).item()
        return "Fake News" if pred == 1 else "Real News"

# Keep Asking for Input

while True:
    user_input = input("Enter text or image path (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    print("Prediction:", predict(user_input))
    



Enter text or image path (or type 'exit' to quit):  டிரம்ப் சாத்தியமான அமைச்சரவை உறுப்பினர்களின் பெயர்களுடன் நிறுத்தவில்லை, அமெரிக்காவை பாதுகாப்பானதாக மாற்ற வடிவமைக்கப்பட்ட கொள்கைகளையும் இரட்டிப்பாக்கினார்.அமெரிக்காவின் தேசிய பாதுகாப்பை அரசியல் சரியான தன்மைக்கு முன் வைக்க டிரம்ப் பரிந்துரைக்கும் போது தாராளவாதிகள் ஏன் இவ்வளவு புண்படுத்தப்படுகிறார்கள் என்று வாக்காளர்கள் கேட்க வேண்டும்? டொனால்ட் டிரம்ப் தனது இரண்டு சர்ச்சைக்குரிய கொள்கை திட்டங்களில் ஒட்டிக்கொண்டிருக்கிறார், இப்போது அவர் குடியரசுக் கட்சியின் ஊக ஜனாதிபதி வேட்பாளராக மாறிவிட்டார்.தீவிரவாதத்தின் ஆபத்துக்கள் காரணமாக ஜனாதிபதியாக தேர்ந்தெடுக்கப்பட்டால், அவர் நாட்டிற்குள் நுழைவதை தற்காலிகமாகப் பட்டி செய்வதற்கான தனது திட்டத்திற்கு அவர் நிற்கிறார் என்று எஸ் லெஸ்டர் ஹோல்ட். அவர் கூறுகையில், நாம் விழிப்புடன் இருக்க வேண்டும்.சட்டவிரோதமாக நாட்டில் வசிக்கும் 11 மில்லியன் மக்கள் அனைவரையும் நாடு கடத்துவதற்கான தனது திட்டத்திலும் டிரம்ப் நிற்கிறார். ஆம், அவர்கள் நாடு கடத்தப்படுவார்கள் என்று அவர் கூறுகிறார்.சிலரைத் திரும்ப அனுமதிக்கும் ஒரு அம

Prediction: Fake News


Enter text or image path (or type 'exit' to quit):  	text 180	kabul (reuters) - afghanistan s telecoms regulator wrote to internet service providers this week ordering them to block the messaging services whatsapp and telegram but it was not immediately clear whether they had complied. use of social media and mobile instant messaging services has exploded in afghanistan over recent years. social media users and civil rights groups reacted with outrage to initial reports of the move and the letter sent by telecoms regulator atra was widely shared on social media.   some media reports, citing unidentified sources, said the move had been ordered by the national directorate for security to thwart the use of the encrypted messaging services by the taliban and other insurgent groups. it was not immediately possible to confirm the reports. the acting minister for telecommunications, shahzad aryobee, posted a message on facebook saying that the telecoms regulator had been ordered to put a grad

Prediction: Real News


Enter text or image path (or type 'exit' to quit):  	text 188	వాషింగ్టన్ (రాయిటర్స్) - ఫెడరల్ హెల్త్‌కేర్ ఇన్సూరెన్స్ కవరేజ్ పొందే అక్రమ వలసదారులను అణిచివేసేందుకు ఉద్దేశించిన నిబంధనను తొలగించడానికి రిపబ్లికన్ నాయకులు ఒబామాకేర్ రోల్‌బ్యాక్ బిల్లును సర్దుబాటు చేసిన తరువాత వాషింగ్టన్లో కొంతమంది సంప్రదాయవాదులు మంగళవారం మంగళవారం మంగళవారం ఫ్యూమ్ అయ్యారు.U.S. లో రిపబ్లికన్ నాయకులకు ఈ అభివృద్ధి మరొక సమస్యను కలిగించింది.ప్రతినిధుల సభ మరియు అధ్యక్షుడు డొనాల్డ్ ట్రంప్, విస్తృత బిల్లుకు కాంగ్రెస్ ఆమోదం పొందటానికి ప్రయత్నిస్తున్నారు, ఇది అధికారం చేపట్టినప్పటి నుండి ట్రంప్‌కు మొదటి పెద్ద శాసన పరీక్ష.హెల్త్‌కేర్ టాక్స్ క్రెడిట్స్ యు.ఎస్.పౌరులు మరియు చట్టపరమైన శాశ్వత నివాసితులు, అక్రమ వలసదారులకు కాదు.బిల్లు ప్రకారం పన్ను క్రెడిట్లను నిర్వహించే సెనేట్ ఫైనాన్స్ కమిటీ, హోంల్యాండ్ సెక్యూరిటీ విభాగంలో పరిధిని లేదని సెనేట్ పార్లమెంటు సభ్యుడు నిర్ధారించడంతో, చాలా విధానపరమైన చర్యలో, సెనేట్ పార్లమెంటు సభ్యుడు నిర్ధారించిన తరువాత దీనిని బిల్లు నుండి తొలగించారు.రిపబ్లికన్ ప్రతినిధి లౌ బార్లెట్టా సోమవారం సాయంత్రం

...

Enter text or image path (or type 'exit' to quit):   C:\Users\golla\Desktop\fake.jpg"


Prediction: Fake News


Enter text or image path (or type 'exit' to quit):   C:\Users\golla\Desktop\real.jpg"


Prediction: Fake News


Enter text or image path (or type 'exit' to quit):  exit


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from transformers import XLMRobertaTokenizer, XLMRobertaModel
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import os
from PIL import Image

In [4]:
# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directories
image_dir = r"C:\Users\golla\Desktop\BTECH\8THSEM"
csv_path = "final_data.csv"

# Image Preprocessing (Fixed)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [5]:
# Load Pre-trained ResNet-50 Model
class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
        for param in self.resnet.parameters():
            param.requires_grad = False  # Freeze all layers except the last one
        self.resnet.fc = nn.Linear(2048, 2)  # Modify last layer for binary classification
    
    def forward(self, x):
        return self.resnet(x)

resnet_model = ResNetModel().to(device)

In [9]:
# Load Pre-trained XLM-RoBERTa Model
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
xlmr_model = XLMRobertaModel.from_pretrained("xlm-roberta-base")

# Define Text Classifier Model
class TextClassifier(nn.Module):
    def __init__(self):
        super(TextClassifier, self).__init__()
        self.xlmr = xlmr_model
        for param in self.xlmr.parameters():
            param.requires_grad = False  # Freeze pre-trained layers
        self.fc = nn.Linear(768, 2)  # Modify last layer
    
    def forward(self, input_ids, attention_mask):
        outputs = self.xlmr(input_ids=input_ids, attention_mask=attention_mask)
        return self.fc(outputs.pooler_output)

text_model = TextClassifier().to(device)



In [10]:
# Define Multimodal Model
class MultiModalModel(nn.Module):
    def __init__(self, resnet_model, text_model):
        super(MultiModalModel, self).__init__()
        self.resnet = resnet_model
        self.text_model = text_model
        self.fc = nn.Linear(4, 2)  # Combining both outputs (2 from image + 2 from text)

    def forward(self, image=None, input_ids=None, attention_mask=None):
        image_output = self.resnet(image) if image is not None else torch.zeros((1, 2)).to(device)
        text_output = self.text_model(input_ids, attention_mask) if input_ids is not None else torch.zeros((1, 2)).to(device)

        combined_output = torch.cat((image_output, text_output), dim=1)
        return self.fc(combined_output)

# Initialize multimodal model
multimodal_model = MultiModalModel(resnet_model, text_model).to(device)


In [17]:
# Define Dataset Class for Multimodal Data
class MultiModalDataset(Dataset):
    def __init__(self, image_folder, csv_path, transform, tokenizer):
        self.image_paths = []
        self.labels = []
        self.texts = []

        # Load image data
        for label, folder in enumerate(["real", "fake"]):
            folder_path = os.path.join(image_folder, folder)
            for filename in os.listdir(folder_path):
                self.image_paths.append(os.path.join(folder_path, filename))
                self.labels.append(label)

        # Load text data
        df = pd.read_csv(csv_path)
        self.texts = df["text"].tolist()
        self.text_labels = df["label"].tolist()

        self.transform = transform
        self.tokenizer = tokenizer

    def __len__(self):
        return max(len(self.image_paths), len(self.texts))  # Ensure full batch support

    def __getitem__(self, idx):
        # Load image
        if idx < len(self.image_paths):
            img = Image.open(self.image_paths[idx]).convert("RGB")
            img = self.transform(img)
            img_label = self.labels[idx]
        else:
            img = torch.zeros((3, 224, 224))  # Placeholder tensor
            img_label = -1  # Ignore in accuracy calculation

        # Load text
        if idx < len(self.texts):
            text = self.texts[idx]
            if not isinstance(text, str):  
                text = ""  # Convert None/NaN to empty string to prevent errors

            encoding = self.tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=256)
            text_label = self.text_labels[idx]
        else:
            encoding = {"input_ids": torch.zeros((1, 256), dtype=torch.long), "attention_mask": torch.zeros((1, 256), dtype=torch.long)}
            text_label = -1  # Ignore in accuracy calculation

        return img, img_label, encoding["input_ids"].squeeze(0), encoding["attention_mask"].squeeze(0), text_label

# Load dataset
test_dataset = MultiModalDataset(os.path.join(image_dir, "test"), csv_path, transform, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [ ]:
# Accuracy Calculation for Multimodal Model
def evaluate_multimodal(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, img_labels, input_ids, attention_mask, text_labels in loader:
            images, img_labels = images.to(device), img_labels.to(device)
            input_ids, attention_mask, text_labels = input_ids.to(device), attention_mask.to(device), text_labels.to(device)

            outputs = model(image=images, input_ids=input_ids, attention_mask=attention_mask)
            predictions = outputs.argmax(1)

            # Consider only valid labels (ignore -1 cases)
            valid_labels = (img_labels >= 0) | (text_labels >= 0)
            true_labels = torch.where(img_labels >= 0, img_labels, text_labels)

            correct += (predictions[valid_labels] == true_labels[valid_labels]).sum().item()
            total += valid_labels.sum().item()

    return correct / total if total > 0 else 0

# Compute Accuracy
multimodal_accuracy = evaluate_multimodal(multimodal_model, test_loader)
print(f"Final Multimodal Model Accuracy: {multimodal_accuracy}")